In [38]:
import pandas as pd
import quandl
import numpy as np

In [27]:
import sys

In [28]:
print (sys.executable)

/Users/salehajaweid/opt/anaconda3/bin/python


In [30]:
#pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [31]:
import yfinance as yf
googl = yf.Ticker("GOOGL")
df = googl.history(period="max")  # gets all available historical data

In [32]:
print (df.head())

                               Open      High       Low     Close     Volume  \
Date                                                                           
2004-08-19 00:00:00-04:00  2.490595  2.591713  2.389974  2.499063  893181924   
2004-08-20 00:00:00-04:00  2.515750  2.716741  2.503048  2.697563  456686856   
2004-08-23 00:00:00-04:00  2.758334  2.826327  2.715994  2.724711  365122512   
2004-08-24 00:00:00-04:00  2.770538  2.779504  2.579509  2.611887  304946748   
2004-08-25 00:00:00-04:00  2.614129  2.689843  2.587231  2.640031  183772044   

                           Dividends  Stock Splits  
Date                                                
2004-08-19 00:00:00-04:00        0.0           0.0  
2004-08-20 00:00:00-04:00        0.0           0.0  
2004-08-23 00:00:00-04:00        0.0           0.0  
2004-08-24 00:00:00-04:00        0.0           0.0  
2004-08-25 00:00:00-04:00        0.0           0.0  


In [5]:
df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
print (df.head())

                               Open      High       Low     Close     Volume
Date                                                                        
2004-08-19 00:00:00-04:00  2.490595  2.591713  2.389974  2.499063  893181924
2004-08-20 00:00:00-04:00  2.515750  2.716741  2.503048  2.697563  456686856
2004-08-23 00:00:00-04:00  2.758335  2.826327  2.715994  2.724711  365122512
2004-08-24 00:00:00-04:00  2.770538  2.779504  2.579509  2.611887  304946748
2004-08-25 00:00:00-04:00  2.614129  2.689843  2.587231  2.640031  183772044


In [33]:
df['HL_PCT'] = (df['High'] - df['Close']) / df['Close'] * 100.0
df['PCT_change'] = (df['Close'] - df['Open']) / df['Open'] * 100.0

df = df[['Close', 'HL_PCT', 'PCT_change', 'Volume']]

print (df.head())

                              Close    HL_PCT  PCT_change     Volume
Date                                                                
2004-08-19 00:00:00-04:00  2.499063  3.707398    0.339978  893181924
2004-08-20 00:00:00-04:00  2.697563  0.710947    7.226981  456686856
2004-08-23 00:00:00-04:00  2.724711  3.729431   -1.218975  365122512
2004-08-24 00:00:00-04:00  2.611887  6.417497   -5.726378  304946748
2004-08-25 00:00:00-04:00  2.640031  1.886791    0.990853  183772044


In [55]:
forecast_col = 'Close'
# treat NAN data as outlier

df.fillna(-99999, inplace=True)

# use regression to create a forecast

import math 

forecast_out = int(math.ceil(0.001*len(df))) # gives you the forecast window for the data set based on the volume of the dataset
df['label'] = df[forecast_col].shift(-forecast_out)
print (df.head())

                              Close    HL_PCT  PCT_change     Volume     label
Date                                                                          
2004-08-19 00:00:00-04:00  2.499063  3.707398    0.339978  893181924  2.687601
2004-08-20 00:00:00-04:00  2.697563  0.710947    7.226981  456686856  2.643766
2004-08-23 00:00:00-04:00  2.724711  3.729431   -1.218975  365122512  2.540657
2004-08-24 00:00:00-04:00  2.611887  6.417497   -5.726378  304946748  2.549622
2004-08-25 00:00:00-04:00  2.640031  1.886791    0.990853  183772044  2.496822


In [56]:
df.dropna(inplace=True)
print(df.head())

                              Close    HL_PCT  PCT_change     Volume     label
Date                                                                          
2004-08-19 00:00:00-04:00  2.499063  3.707398    0.339978  893181924  2.687601
2004-08-20 00:00:00-04:00  2.697563  0.710947    7.226981  456686856  2.643766
2004-08-23 00:00:00-04:00  2.724711  3.729431   -1.218975  365122512  2.540657
2004-08-24 00:00:00-04:00  2.611887  6.417497   -5.726378  304946748  2.549622
2004-08-25 00:00:00-04:00  2.640031  1.886791    0.990853  183772044  2.496822


In [57]:
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split  # This replaces cross_validation
from sklearn.linear_model import LinearRegression

In [58]:
# Define features 'x' and labels 'y'

x = np.array(df.drop(['label'], 1))
y = np.array(df['label'])

x = preprocessing.scale(x)

# x = x[:-forecast_out+1]
df.dropna(inplace=True)
y = np.array(df['label'])

print(len(x), len(y))

4704 4704


/var/folders/h4/fh6pmpz12db8s8wlgw7ncgvr0000gn/T/ipykernel_65050/4239986564.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  x = np.array(df.drop(['label'], 1))
/Users/salehajaweid/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:235: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(


In [59]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

In [60]:
clf = LinearRegression()
clf.fit(x_train, y_train)
accuracy = clf.score(x_test, y_test)

print(accuracy)

0.9971178932218895


In [61]:
print(forecast_out)

5


In [63]:
print(df.tail())

                                Close    HL_PCT  PCT_change    Volume  \
Date                                                                    
2023-04-20 00:00:00-04:00  104.789009  0.911767    1.328070  27820800   
2023-04-21 00:00:00-04:00  104.908447  0.559716   -0.056886  25800100   
2023-04-24 00:00:00-04:00  105.465782  0.622814    0.455023  23542800   
2023-04-25 00:00:00-04:00  103.355865  2.734717   -1.870928  46664100   
2023-04-26 00:00:00-04:00  103.216530  2.545559   -1.153259  53347600   

                                label  
Date                                   
2023-04-20 00:00:00-04:00  107.078056  
2023-04-21 00:00:00-04:00  106.829247  
2023-04-24 00:00:00-04:00  106.689911  
2023-04-25 00:00:00-04:00  104.818863  
2023-04-26 00:00:00-04:00  104.908447  
